In [91]:
import pandas as pd
from bs4 import BeautifulSoup
import json
import re


In [89]:
path = "/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/dictionnaire_new-0-200.json"

def digimonLoad(petName):
    dStat = {}
    with open(petName) as f:
        pet = json.loads(f.read())
    return pet
    
data = digimonLoad(path)

In [90]:
def extract_med_info(match):
     
    soup = BeautifulSoup(match, 'html.parser')
    # Créer un dictionnaire pour les données
    data = soup.text.split('\n') 
    button_data = {
        "Nom": data[1].split(':')[0].strip(), 
        "caracteristique": data[1].split(':')[1].strip(), 
        "Code EAN-13" : data[3].split(':')[1].strip(),
        "Médicament GENERIQUE" : data[5].split(' ')[2].strip()
    }
    return button_data
def extract_prices(match):
    match= match.replace("Substance active (DCI) &amp; dosage", "Substance dosage") 
    soup = BeautifulSoup(match, 'html.parser')

    # Extraire les informations et structurer en JSON
    price_officine = soup.find('div', class_='text-danger').get_text(strip=True)
    price_public = soup.find(string="Prix Public de Vente").find_next('div').get_text(strip=True)
    price_remboursement = soup.find(string="Prix base remboursement").find_next('div').get_text(strip=True)
    price_hospitalier = soup.find(string="Prix Hospitalier").find_next('div').get_text(strip=True)
    substance_active = soup.find(string="Substance dosage ").find_next('div').get_text(strip=True)
    classe_therapeutique = soup.find(string="Classe Thérapeutique").find_next('div').get_text(strip=True)

    # Créer un dictionnaire pour les données
    div_data = {
        "Prix Officine": price_officine,
        "Prix Public de Vente": price_public,
        "Prix base remboursement": price_remboursement,
        "Prix Hospitalier": price_hospitalier,
        "Substance active (DCI) & dosage": substance_active,
        "Classe Thérapeutique": classe_therapeutique
    }
    return div_data

In [105]:
def get_matches(name):
    pattern_button = r'<button[^>]*class="btn btn-danger btn-block mt-1 text-left"[^>]*>.*?</button>' 
    # Extraction des balises correspondantes
    matches_b = re.findall(pattern_button, data['CHEMOSET'][1], re.DOTALL) 
    matches_d = []
    for i in range(len(matches_b)-1):
        index_1 = data['CHEMOSET'][1].find(matches_b[i])+len(matches_b[i])
        index_2 = data['CHEMOSET'][1].find(matches_b[i+1])
        matches_d.append(data['CHEMOSET'][1][index_1:index_2])
    index_1 = data['CHEMOSET'][1].find(matches_b[-1])+len(matches_b[-1])
    index_2 = data['CHEMOSET'][1].find('</div></div><div _ngcontent-tkn-c49="" class="col-xl-6 col-lg-6">')
    matches_d.append(data['CHEMOSET'][1][index_1:index_2])
    return matches_b, matches_d

In [109]:
medicaments = []
for i in data: 
    matches_b, matches_d = get_matches(i)
    for match_b,match_d in zip(matches_b, matches_d):
        b_json = extract_med_info(match_b)
        d_json = extract_prices(match_d) 
        b_json.update(d_json)
        medicaments.append(b_json)



In [113]:
df = pd.DataFrame(medicaments)
    

In [117]:
df['Prix Public de Vente'] = df['Prix Public de Vente'].str.replace(' DH', '').astype(float)
df['Prix base remboursement'] = df['Prix base remboursement'].str.replace(' DH', '').astype(float)
df['Prix Hospitalier'] = df['Prix Hospitalier'].str.replace(' DH', '').astype(float)

In [165]:
df.to_csv("/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/scrapedData/medicament-0-200.csv")

In [122]:
df.columns

Index(['Nom', 'caracteristique', 'Code EAN-13', 'Médicament GENERIQUE',
       'Prix Officine', 'Prix Public de Vente', 'Prix base remboursement',
       'Prix Hospitalier', 'Substance active (DCI) & dosage',
       'Classe Thérapeutique'],
      dtype='object')

In [1]:
from pathlib import Path 
import sqlite3
 
 

Tracpharmic = Path.home()/"Tracpharmic"

images = Tracpharmic/"images"

dataset = Tracpharmic/"dataset"/"pharmadataset.db"


name_application = "TracPharmic"  

In [10]:
def create_table_medicament():
    with sqlite3.connect( dataset) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS Medicament (
                ID_Medicament INTEGER PRIMARY KEY AUTOINCREMENT,
                Nom TEXT,
                caracteristique TEXT,
                Code_EAN_13 TEXT UNIQUE,
                Medicament_GENERIQUE TEXT,
                Prix_Officine TEXT,
                Prix_Public_de_Vente REAL,
                Prix_base_remboursement REAL,
                Prix_Hospitalier REAL,
                Substance_active_DCI TEXT,
                Classe_Therapeutique TEXT
            )
        """)
def drop_table_medicament():
    with sqlite3.connect( dataset) as conn:
        cursor = conn.cursor()
        cursor.execute("""DROP TABLE Medicament; 
        """)


def ajouter_medicament(nom, caracteristique, code_ean_13, medicament_generique, prix_officine, prix_public_de_vente,
                       prix_base_remboursement, prix_hospitalier, substance_active_dci, classe_therapeutique):
    with sqlite3.connect( dataset) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO Medicament (
                Nom, caracteristique, Code_EAN_13, Medicament_GENERIQUE, Prix_Officine, Prix_Public_de_Vente,
                Prix_base_remboursement, Prix_Hospitalier, Substance_active_DCI, Classe_Therapeutique
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (nom, caracteristique, code_ean_13, medicament_generique, prix_officine, prix_public_de_vente,
              prix_base_remboursement, prix_hospitalier, substance_active_dci, classe_therapeutique))

In [3]:
import pandas as df

In [12]:
data = df.read_csv("/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/scrapedData/medicament-0-200.csv")

In [13]:
create_table_medicament()

In [14]:
data = data.drop(columns=['Unnamed: 0.1',"Unnamed: 0"])

In [11]:
drop_table_medicament()

In [15]:
data = data.drop_duplicates(subset='Code EAN-13')

In [16]:
for index, row in  data.iterrows():
    item = []
    for i in row.keys():
        
        if 'Unnamed: 0' != i: 
            item.append(row[i])
    ajouter_medicament(item[0],item[1],item[2],item[3],item[4],item[5],item[6],item[7],item[8],item[9])
    
    
    

In [17]:
data

,Nom,caracteristique,Code EAN-13,Médicament GENERIQUE,Prix Officine,Prix Public de Vente,Prix base remboursement,Prix Hospitalier,Substance active (DCI) & dosage,Classe Thérapeutique
0,CHEMOSET,1 BOITE 10 COMPRIME PELLICULE COMPRIME PELLICU...,6118001250585,Remboursable,Prix Officine ( Pharmacie ),175.7,175.7,109.8,ONDANSETRON,ANTIEMETIQUE
1,CHEMOSET,1 BOITE 10 COMPRIME PELLICULE COMPRIME PELLICU...,6118001250615,Remboursable,Prix Officine ( Pharmacie ),175.7,175.7,109.8,ONDANSETRON,ANTIEMETIQUE
2,CHEMOSET,1 BOITE 1 COMPRIME PELLICULE COMPRIME PELLICUL...,6118001250592,Remboursable,Prix Officine ( Pharmacie ),175.7,175.7,109.8,ONDANSETRON,ANTIEMETIQUE
3,CHEMOSET,1 BOITE 5 COMPRIME PELLICULE COMPRIME PELLICUL...,6118001250608,Remboursable,Prix Officine ( Pharmacie ),175.7,175.7,109.8,ONDANSETRON,ANTIEMETIQUE
4,CHEMOSET 2MG-ML,5 AMPOULE 4 ML SOLUTION INJECTABLE à 8 MG,6118001250530,Remboursable,Prix Officine ( Pharmacie ),175.7,175.7,109.8,ONDANSETRON,ANTIEMETIQUE
5,CHEMOSET,1 BOITE 5 COMPRIME PELLICULE COMPRIME PELLICUL...,6118001250578,Remboursable,Prix Officine ( Pharmacie ),175.7,175.7,109.8,ONDANSETRON,ANTIEMETIQUE
6,CHEMOSET,1 BOITE 1 COMPRIME PELLICULE COMPRIME PELLICUL...,6118001250561,Remboursable,Prix Officine ( Pharmacie ),175.7,175.7,109.8,ONDANSETRON,ANTIEMETIQUE
7,CHEMOSET 2MG-ML,5 AMPOULE 2 ML SOLUTION INJECTABLE à 4 MG,6118001250547,Remboursable,Prix Officine ( Pharmacie ),175.7,175.7,109.8,ONDANSETRON,ANTIEMETIQUE
